In [138]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d, Circle, Line, TapTool, CustomJS
from bokeh.plotting import figure
import plotly.graph_objects as go
import plotly.offline as pyo

In [78]:
df = pd.read_csv("Sample - Superstore.csv", index_col = 'Row ID', encoding='cp1252')
df.head(3)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


## Biểu đồ 1

**Tiêu đề:** Top 10 thành phố có tổng lượng mua hàng cao nhất.

**Loại biểu đồ:** Lollipop chart

**Lý do lựa chọn:** Thể hiện rõ sự so sánh tổng lượng mua hàng của các thành phố 

**Trực quan hóa:**

In [79]:

chart_df = df.copy()
top_10_cities = chart_df['City'].value_counts().nlargest(10)
top_10_cities.sort_values(ascending=True, inplace=True)


# Enable inline visualization in Jupyter Notebook
output_notebook(hide_banner=True)

# Define the data for the lollipop chart
x_values = list(top_10_cities.values)
y_values = np.arange(1,11)
# Calculate the x and y coordinates for the line
line_x = [0] * len(x_values)
line_y = [y_values[0]] * len(y_values)
colors = ['#F7D060','#ADE4DB','#957777','#FF6969','#394867','#A5C0DD','#F79540', '#B71375', '#B71375', '#8B1874']

# Add the x and y coordinates for the line to the data dictionary
data = {'x': x_values, 'y': y_values, 'line_x': line_x, 'line_y': line_y, 'colors': colors}

# Create a ColumnDataSource object to hold the data
source = ColumnDataSource(data)

# Create a new figure
y_range = Range1d(start=0, end=11)
p = figure(height=600, width=800, title='Top 10 most buying cities', y_range=y_range, tools='tap')
p.yaxis.ticker = y_values
p.yaxis.major_label_overrides = dict(zip(y_values, top_10_cities.index))


# Draw the horizontal lollipop chart
glyph = Circle(x='x', y='y', size=10, fill_color='colors', line_color='white')
renderer = p.add_glyph(source, glyph)
p.segment(x0=0, y0='y', x1='x', y1='y',line_color='colors', source=source)

line_glyph = Line(x='line_x', y='line_y', line_color='black', line_width=3, line_alpha=1)
p.add_glyph(source, line_glyph)

# Add tooltips to display the data
hover = HoverTool(tooltips=[('X', '@x'), ('Y', '@y')], renderers=[renderer])
p.add_tools(hover)

# Define the callback function to handle click events
callback = CustomJS(args=dict(line_glyph=line_glyph), code="""
    var selected_index = cb_obj.indices[0];
    var selected_glyph = cb_obj.selection_glyph;
    selected_glyph.size = 20;
    selected_glyph.line_color = 'red';
    line_glyph.line_alpha = 1;
    line_glyph.data_source.data.line_x = [0, cb_obj.data.x[selected_index]];
    line_glyph.data_source.data.line_y = [cb_obj.data.y[selected_index], cb_obj.data.y[selected_index]];
    line_glyph.data_source.change.emit();
""")

# Add the callback function to the TapTool
taptool = p.select(type=TapTool)
taptool.callback = callback

# Display the chart
show(p)

**Kỹ thuật trực quan sử dụng:**

- ***Kỹ thuật Reduce:*** Để người dùng dễ dàng tập trung vào thành phố cần quan tâm. Chẳng hạn như thành phố có tổng lượng mua cao nhất, thành phố có tổng lượng mua thấp nhất trong top 10 thành phố.

=> Reduce làm giảm thiểu sự phức tạp, tránh gây rối mắt cho người xem, làm tăng tính thể hiện cho biểu đồ.


**Nhận xét:**
- Hai thành phố có tổng lượng mua cao nhất là New York và Los Angeles.
- Hai thành phố có tổng lượng mua thấp nhất là San Diego và Springfield

## Biểu đồ 2

**Tiêu đề:** Mối quan hệ giữa discount và profit ở từng năm (2014-2017).

**Loại biểu đồ:** Horizontal Bar chart.

**Lý do lựa chọn:** Thể hiện rõ được mối liên hệ giữa hai yếu tố.

**Trực quan hóa:**

In [137]:
#Process data
temp_df = df.copy()
temp_df['Order Date'] = pd.to_datetime(temp_df['Order Date'])

discount_label = temp_df['Discount'].unique()
categories = ["%"] * len(plot_df)
discount_label.sort(axis=0)
for i in range(len(discount_label)):
    categories[i] = str(int(discount_label[i] * 100)) + categories[i]

data = {}
for i in range(2014, 2018):
    _df = temp_df[temp_df['Order Date'].dt.year == i].groupby(['Discount'])['Profit'].sum()
    values = [0] * len(_df)
    for j in range(len(_df)):
        values[j] = round(_df.values[j], 2)
    data['profit_' + str(i)] = values
plot_df = pd.DataFrame(data, index=categories)



# Map positive and negative values to colors
colors = ['#C68387' if v < 0 else '#A4D0A4' for v in plot_df['profit_2014'].values]

# Create trace
trace = go.Bar(
    x=list(plot_df['profit_2014'].values),
    y=list(plot_df['profit_2014'].index),
    orientation='h',
    marker=dict(
        color=colors
    ),
    text=values,
    textposition="auto",
    texttemplate='<b>%{text}</b>',
    textangle = 0
)

# Create layout
layout = go.Layout(
    title = '<b>The effect of discount on profit in each year (2014-2017)</b>',
    xaxis=dict(title='Profit'),
    yaxis=dict(
        title='Discount',
        tickvals=list(range(len(list(plot_df['profit_2014'].index)))),
        ticktext=list(plot_df['profit_2014'].index)
    ),
    height=800, # Adjust height to fit y-tick labels
    width=800,   # Adjust width as needed
    margin = dict( t = 100, b = 50, r = 70, l = 75),
    font = dict(
            family = "Arial",
            size = 10,
            color= '#101010',
                    ),
    titlefont = dict(
            family =  "Arial",#"Overpass",
            size = 20,
            color = '#101010'
                    ),
    
)

buttons = []
for i in range(2014, 2018):
    profit = list(plot_df['profit_' + str(i)].values)
    discount = list(plot_df['profit_' + str(i)].index)
    colors = ['#C68387' if v < 0 else '#A4D0A4' for v in profit]
    _title = '<b>The effect of discount on profit in ' + str(i) + '</b>'
    _dict = dict(
        args=[{'x': [profit], 'y': [discount], 'marker.color': [colors], 'title': _title, 
                       'yaxis.tickvals': list(range(len(discount))), 'yaxis.ticktext': discount}],
                label=str(i),
                method='update'
    )
    buttons.append(_dict)

update_menu = list([
    dict(
    buttons = buttons,
    direction='down',
        showactive=True,
        x=1,
        xanchor='left',
        y=1.1,
        yanchor='top'

    )
])

layout['updatemenus'] = update_menu
# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show figure
pyo.iplot(fig)


**Kỹ thuật trực quan sử dụng:**

- ***Màu sắc***: dùng màu sắc để thể hiện cho lợi nhuận thu được ở mỗi mức giảm giá là mức dương hay mức âm:
    - Màu xanh: lợi nhuận dương.
    - Màu đỏ: lợi nhuận âm.
- ***Kỹ thuật Manipulate View:*** cho kép người dùng thực hiện các thao tác trực tiếp trên biểu đồ để thay đổi hiển thị dữ liệu. Ở đây biểu đồ này cho phép người dùng sử dụng bộ lọc (filter) để xem mối liên hệ giữa lợi nhuận thu được và chiến dịch giảm giá theo từng năm

**Nhận xét:**
- Ta thấy rõ mối liên hệ giữa profit và discount ở các năm đều là tỷ lệ nghịch. Khi khuyến mãi càng tăng thì lợi nhuận càng giảm thậm chí dẫn đến lỗ.
- Tuy nhiên, cần phải duy trì các chiến dịch khuyến mãi ở mức vừa phải để kích thích nhu cầu mua sắm và tăng mức độ nhận diện thương hiệu cho cửa hàng.